In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.layers import Activation
from keras.layers import Conv2D, MaxPooling2D, Input
from keras.layers import BatchNormalization
import tensorflow as tf
from keras.datasets import cifar10
from keras.models import Model, Sequential


2025-11-17 18:06:16.600908: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-17 18:06:16.630559: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-17 18:06:17.580869: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-17 18:06:21.747300: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To tur

In [2]:
# Download data
(train_data, train_labels), (test_data, test_labels) = cifar10.load_data()

# Normalize inputs
train_data = train_data.astype('float32') / 255.
test_data = test_data.astype('float32') / 255.

# One-hot output vectors
train_labels_onehot = tf.keras.utils.to_categorical(train_labels, 10)
test_labels_onehot = tf.keras.utils.to_categorical(test_labels, 10)

filename = "CIFAR_v17_vgg.keras"

In [3]:
def define_functional_model():
    inputs = Input(shape=(32, 32, 3), name="input_layer")
    x = Conv2D(32, (3, 3), use_bias=False, activation='relu', padding='same', name='block1_conv1')(inputs)
    x = Conv2D(32, (3, 3), use_bias=False, activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), name='block1_pool')(x)
    x = Dropout(0.2, name='block1_dropout')(x)
    x = Conv2D(64, (3, 3), use_bias=False, activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(64, (3, 3), use_bias=False, activation='relu', padding='same', name='block2_conv2')(x)
    x = Conv2D(64, (3, 3), use_bias=False, activation='relu', padding='same', name='block2_conv3')(x)
    x = MaxPooling2D((2, 2), name='block2_pool')(x)
    x = Dropout(0.2, name='block2_dropout')(x)
    x = Conv2D(128, (3, 3), use_bias=False, activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(128, (3, 3), use_bias=False, activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(128, (3, 3), use_bias=False, activation='relu', padding='same', name='block3_conv3')(x)
    x = MaxPooling2D((2, 2), name='block3_pool')(x)
    x = Dropout(0.2, name='block3_dropout')(x)
    x = Flatten(name='flatten')(x)
    x = Dense(128, use_bias=False, activation='relu', kernel_initializer='he_uniform', name='fc1')(x)
    x = Dropout(0.2, name='fc1_dropout')(x)
    outputs = Dense(10, use_bias=False, activation='softmax', name='predictions')(x)
    
    model = Model(inputs=inputs, outputs=outputs, name="cifar10_functional_cnn")
    
    opt = SGD(learning_rate=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

model = define_functional_model()
model.summary()


2025-11-17 18:06:28.865604: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "cifar10_functional_cnn"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 32, 32, 32)     │           864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 32, 32, 32)     │         9,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_dropout (Dropout)        │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 16, 16, 64)     │        18,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 16, 16, 64)     │        36,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv3 (Conv2D)           │ (None, 16, 16, 64)     │        36,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_dropout (Dropout)        │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 8, 8, 128)      │        73,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 8, 8, 128)      │       147,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 8, 8, 128)      │       147,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_dropout (Dropout)        │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc1 (Dense)                     │ (None, 128)            │       262,144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc1_dropout (Dropout)           │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ predictions (Dense)             │ (None, 10)             │         1,280 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 734,304 (2.80 MB)

 Trainable params: 734,304 (2.80 MB)

 Non-trainable params: 0 (0.00 B)

In [4]:

# Train model (use one-hot labels)
history = model.fit(
    train_data, train_labels_onehot,       # training data (one-hot labels)
    batch_size=64,                        # batch size
    epochs=100,                            # Maximum number of epochs
    validation_split=0.1,                  # Percentage of training data used for validation
)

# Test model: get class predictions and evaluate with one-hot labels
predictions_keras = np.argmax(model.predict(test_data, verbose=0), axis=1)
test_loss, test_accuracy = model.evaluate(test_data, test_labels_onehot, verbose=0)
print(f"Test accuracy: {test_accuracy}")
model.save(filename)

Epoch 1/100
704/704 ━━━━━━━━━━━━━━━━━━━━ 74s 101ms/step - accuracy: 0.1103 - loss: 2.2998 - val_accuracy: 0.1132 - val_loss: 2.2892
Epoch 2/100
704/704 ━━━━━━━━━━━━━━━━━━━━ 69s 99ms/step - accuracy: 0.1574 - loss: 2.2345 - val_accuracy: 0.2052 - val_loss: 2.1758
Epoch 3/100
704/704 ━━━━━━━━━━━━━━━━━━━━ 84s 119ms/step - accuracy: 0.2348 - loss: 2.0620 - val_accuracy: 0.3008 - val_loss: 1.9276
Epoch 4/100
704/704 ━━━━━━━━━━━━━━━━━━━━ 98s 139ms/step - accuracy: 0.3067 - loss: 1.8942 - val_accuracy: 0.3356 - val_loss: 1.8249
Epoch 5/100
704/704 ━━━━━━━━━━━━━━━━━━━━ 110s 157ms/step - accuracy: 0.3528 - loss: 1.7763 - val_accuracy: 0.4010 - val_loss: 1.6465
Epoch 6/100
704/704 ━━━━━━━━━━━━━━━━━━━━ 90s 127ms/step - accuracy: 0.3925 - loss: 1.6694 - val_accuracy: 0.4328 - val_loss: 1.5775
Epoch 7/100
704/704 ━━━━━━━━━━━━━━━━━━━━ 90s 127ms/step - accuracy: 0.4227 - loss: 1.5962 - val_accuracy: 0.4620 - val_loss: 1.4960
Epoch 8/100
704/704 ━━━━━━━━━━━━━━━━━━━━ 85s 121ms/step - accuracy: 0.4470 -

In [5]:
import ANNarchy
from ANNarchy.extensions.ann_to_snn_conversion import ANNtoSNNConverter
ANNarchy.clear()
snn_converter = ANNtoSNNConverter(
    input_encoding='IB', 
    hidden_neuron='IaF',
    read_out='spike_count',
)


ANNarchy 4.8 (4.8.2.5) on linux (posix).


In [6]:
net = snn_converter.load_keras_model(filename, show_info=True)

* Input layer: input_layer, (32, 32, 3)
* InputLayer skipped.
* Conv2D layer: block1_conv1, (32, 32, 32) 
* Conv2D layer: block1_conv2, (32, 32, 32) 
* MaxPooling2D layer: block1_pool, (16, 16, 32) 
* Dropout skipped.
* Conv2D layer: block2_conv1, (16, 16, 64) 
* Conv2D layer: block2_conv2, (16, 16, 64) 
* Conv2D layer: block2_conv3, (16, 16, 64) 
* MaxPooling2D layer: block2_pool, (8, 8, 64) 
* Dropout skipped.
* Conv2D layer: block3_conv1, (8, 8, 128) 
* Conv2D layer: block3_conv2, (8, 8, 128) 
* Conv2D layer: block3_conv3, (8, 8, 128) 
* MaxPooling2D layer: block3_pool, (4, 4, 128) 
* Dropout skipped.
* Flatten skipped.
* Dense layer: fc1, 128 
    weights: (128, 2048)
    mean -0.0014412538148462772, std 0.03463873267173767
    min -0.1399230659008026, max 0.14790278673171997
* Dropout skipped.
* Dense layer: predictions, 10 
    weights: (10, 128)
    mean 0.00020886892161797732, std 0.17771854996681213
    min -0.39750248193740845, max 0.38442927598953247



In [7]:
predictions_snn = snn_converter.predict(test_data[:300], duration_per_sample=200)

100%|██████████| 300/300 [1:20:43<00:00, 16.14s/it]


In [8]:
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

print(classification_report(test_labels[:300], predictions_snn))
print("Test accuracy of the SNN:", accuracy_score(test_labels[:300], predictions_snn))

              precision    recall  f1-score   support

           0       0.15      0.11      0.13        36
           1       0.19      0.33      0.24        24
           2       0.11      0.11      0.11        27
           3       0.03      0.03      0.03        29
           4       0.14      0.13      0.14        23
           5       0.12      0.11      0.11        28
           6       0.23      0.18      0.20        34
           7       0.11      0.15      0.13        27
           8       0.17      0.16      0.17        37
           9       0.24      0.20      0.22        35

    accuracy                           0.15       300
   macro avg       0.15      0.15      0.15       300
weighted avg       0.15      0.15      0.15       300

Test accuracy of the SNN: 0.15


In [9]:
model = tf.keras.models.load_model(filename)
loss, accuracy = model.evaluate(test_data, test_labels_onehot, verbose=1)

313/313 ━━━━━━━━━━━━━━━━━━━━ 7s 20ms/step - accuracy: 0.7905 - loss: 0.8257


In [10]:
print(f"Test accuracy of the ANN: {accuracy:.4f}")
print("Test accuracy of the SNN:", accuracy_score(test_labels[:300], predictions_snn))

Test accuracy of the ANN: 0.7905
Test accuracy of the SNN: 0.15
